In [3]:
from typing import Tuple

import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme()
from tensorflow import keras

from constants import TRAINING_PARTITIONS, ALL_PARTITIONS, annotation_mapping, amino_acid_mapping, reverse_annotation_mapping
from constants import TYPES, KINGDOMS, METRIC_KINGDOMS, METRIC_TYPES
from metrics.metrics import *
from utils.Dataset import Dataset
from utils.helpers import getDatasetPath
from utils.encoding import categoricalToSequence, oneHotToCategorical, sequenceToCategorical, categoricalToOneHot
from serialization import Serializer

## Load cross-validation models

In [5]:
# Load models
run_timestamp = "20211113-0233"
base_path = f"../results/{run_timestamp}/"
cv_models = {}
for k in TRAINING_PARTITIONS:
    cv_models[k] = keras.models.load_model(base_path + f"models/holdout-fold_{k}_final.h5")

# Load logged metrics
logged_metrics = {}
for k in TRAINING_PARTITIONS:
    logged_metrics[k] = pd.read_csv(
        base_path + f"metrics/holdout-fold_{k}.csv",
        index_col="epoch"
    )

cv_training_metrics = pd.concat([df for df in logged_metrics.values()], keys=[k for k in logged_metrics.keys()])
cv_training_metrics.index.rename(["holdout_fold", "epoch"], inplace=True)
cv_training_metrics = cv_training_metrics.reset_index()
cv_training_metrics.set_index(["holdout_fold"], inplace=True)

# 1-based indexing for epochs
cv_training_metrics["epoch"] = cv_training_metrics["epoch"] + 1

# Evaluation on Test Set

## Load test set

In [7]:
dataset = Dataset(getDatasetPath())


## Make predictions

In [8]:
dataframes = {}
for k in TRAINING_PARTITIONS:
    test_data = dataset.getFolds([k])
    test_x = np.array([categoricalToOneHot(sequenceToCategorical(seq, amino_acid_mapping), amino_acid_mapping) for seq in test_data["sequence"]])
    predictions = cv_models[k].predict(test_x)
    dataframes[k] = test_data.copy(deep=True)
    dataframes[k]["prediction"] = np.array([categoricalToSequence(oneHotToCategorical(pred), reverse_annotation_mapping) for pred in predictions])

data = pd.concat([df for df in dataframes.values()], keys=[k for k in dataframes.keys()])
data.index = data.index.droplevel(0)

In [14]:
data

sequence  \
partition type kingdom  number                                                      
1         SP   EUKARYA  0       MKIILTLSIFLICFLQLGQSVIDPSQNEVMSDLLFNLYGYDKSLDP...   
                        1       METVLILCSLLAPVVLASAAEKEKEKDPFYYDYQTLRIGGLVFAVV...   
                        2       MKIILILSIFLICFLQLGQSVIDPSQNEVMSDLLFNLYGYDKSLDP...   
                        3       MKSVQFCFLFCCWRAICCRSCELTNITITVEKEECSFCISINTTWC...   
                        4       MKTLQFFFLFCCWKAICCNSCELTNITIAIEKEECRFCISINTTWC...   
...                                                                           ...   
4         LIPO NEGATIVE 201     MKGSVRALCAFLGVGALGSALCVSCTTVCPHAGKAKAEKVECALKG...   
                        202     MKLVHMASGLAVAIALAACADKSADIQTPAPAANTSISATQQPAIQ...   
                        203     MRLLIGFALALALIGCAQKGAESIGSQKENDLNLEDSSKKSHQNAK...   
                        204     MKRSFLSLAVAAVVLSGCSLIPDYQRPEAPVAAAYPQGQAYGQNTG...   
                        205     MEMLKFGKFAALALAMAVAVGCSSKGGDASGEGANGGVDPNAGYGA...   

                                                                       annotation  \
partition type kingdom  number                                                      
1         SP   EUKARYA  0       SSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOO...   
                        1       SSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOMMMMMMMMMMMM...   
                        2       SSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOO...   
                        3       SSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOOO...   
                        4       SSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOOOO...   
...                                                                           ...   
4         LIPO NEGATIVE 201     LLLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOO...   
                        202     LLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOO...   
                        203     LLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...   
                        204     LLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...   
                        205     LLLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOO...   

                                                                       prediction  
partition type kingdom  number                                                     
1         SP   EUKARYA  0       SSSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOO...  
                        1       ISSMMMMMMMMMMMMMMMMOOOOOOOOOOOOOOOIMMMMMMMMMMM...  
                        2       SSSSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOO...  
                        3       IISSSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOO...  
                        4       SSSSSSSSSSSSSSSSSOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...  
...                                                                           ...  
4         LIPO NEGATIVE 201     LLLLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOO...  
                        202     LLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOO...  
                        203     SLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...  
                        204     LLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...  
                        205     LLLLLLLLLLLLLLLLLLLLOOOOOOOOOOOOOOOOOOOOOOOOOO...  

[16467 rows x 3 columns]

## Compute stratified metrics

In [9]:
def getRelevantData(query: str) -> Tuple[np.ndarray, np.ndarray]:
    relevant_data = data.query(query)
    y_pred = np.array([sequenceToCategorical(seq, annotation_mapping) for seq in relevant_data["prediction"]])
    y_true = np.array([sequenceToCategorical(seq, annotation_mapping) for seq in relevant_data["annotation"]])

    return (y_pred, y_true)

In [11]:
metrics = [MCC(), Recall("macro"), Precision("macro"), Accuracy()]
final_metrics_dict = {}

for k in TRAINING_PARTITIONS:
    final_metrics_dict[k] = {}
    for metric in metrics:
        # All sequences
        y_pred, y_true = getRelevantData(f"partition == {k}")
        final_metrics_dict[k][metric.name] = {
            "overall": {
                "overall": metric(y_true, y_pred)
            }
        }

        # By kingdom
        for kingdom in KINGDOMS:
            y_pred, y_true = getRelevantData(f"kingdom == '{kingdom}' and partition == {k}")
            final_metrics_dict[k][metric.name]["overall"][kingdom] = metric(y_true, y_pred)

        # By pathway
        for type in TYPES:
            y_pred, y_true = getRelevantData(f"type == '{type}' and partition == {k}")
            final_metrics_dict[k][metric.name][type] = {"overall": metric(y_true, y_pred)}

        # By pathway & type
        for type in TYPES:
            for kingdom in KINGDOMS:
                y_pred, y_true = getRelevantData(f"kingdom == '{kingdom}' and type == '{type}' and partition == {k}")
                final_metrics_dict[k][metric.name][type][kingdom] = metric(y_true, y_pred)

# Manually exclude non-existing labels (L & T) from eukarya
for k in TRAINING_PARTITIONS:
    y_pred, y_true = getRelevantData("kingdom == 'EUKARYA'")
    final_metrics_dict[k]["precision"]["overall"]["EUKARYA"] = precision_score(y_true.flatten(), y_pred.flatten(), average="macro", labels=[0,3,4,5])
    final_metrics_dict[k]["recall"]["overall"]["EUKARYA"] = recall_score(y_true.flatten(), y_pred.flatten(), average="macro", labels=[0,3,4,5])

# Convert to dataframe
cv_final_metrics = pd.DataFrame([
    (partition, metric.name, type, kingdom, final_metrics_dict[partition][metric.name][type][kingdom])
    for partition in TRAINING_PARTITIONS
    for metric in metrics
    for type in METRIC_TYPES
    for kingdom in METRIC_KINGDOMS
])

cv_final_metrics.columns = ["holdout_fold", "metric", "type", "kingdom", "value"]
cv_final_metrics.set_index(["holdout_fold", "metric", "type", "kingdom"], inplace=True)

C:\Users\juliu\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\juliu\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\juliu\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\juliu\AppData\Local\Pr

In [19]:
cv_final_metrics.query("kingdom == 'overall' and metric == 'recall'")

value    0.481761
dtype: float64

0.4817613871171912

value
holdout_fold metric type    kingdom          
1            recall LIPO    overall  0.561048
                    TAT     overall  0.257389
                    overall overall  0.840306
                    NO_SP   overall  0.396144
                    SP      overall  0.451703
2            recall LIPO    overall  0.558404
                    TAT     overall  0.290026
                    overall overall  0.867915
                    NO_SP   overall  0.385453
                    SP      overall  0.295481
3            recall LIPO    overall  0.508710
                    TAT     overall  0.289389
                    overall overall  0.872006
                    NO_SP   overall  0.402390
                    SP      overall  0.330215
4            recall LIPO    overall  0.412983
                    TAT     overall  0.285457
                    overall overall  0.860621
                    NO_SP   overall  0.394724
                    SP      overall  0.374865

KeyError: 'values'

# Store results

In [12]:
Serializer.save(cv_final_metrics, "cv_metrics")
Serializer.save(cv_training_metrics, "cv_training_metrics")